## Start the NLP server


In [246]:
import os

STANFORD_NLP_LOCATION = '~/stanford-corenlp-full-2018-10-05/' # the location of the Stanford NLP library on my computer 
                                                              #(download at https://stanfordnlp.github.io/CoreNLP)
STANFORD_NLP_TIMEOUT = 10000 # the time after which a given NLP request will be killed if not yet complete

exec = os.popen
exec('cd %s; java -mx5g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -timeout %s' % 
     (STANFORD_NLP_LOCATION, STANFORD_NLP_TIMEOUT))

## Create function to determine the sentiment of a block of text

In [244]:
from pycorenlp import StanfordCoreNLP
import re
import numpy

# Determines the sentiment of a given block of text as a range from 0 (Extremely Negative) to 4 (Extremely Positive).
def findSentiment(text):
    NLP_SERVER_LOCATION = 'http://localhost:9000'
    PROPERTIES_DICTIONARY = {'annotators': 'sentiment', 'outputFormat': 'json', 'timeout': 1000}
    
    nlp = StanfordCoreNLP(NLP_SERVER_LOCATION)
    result = nlp.annotate(text, properties = PROPERTIES_DICTIONARY)

    sentiments = []
    for sentenceAnalysis in result['sentences']:
        sentence = " ".join(t['word'] for t in sentenceAnalysis['tokens'])
        sentence = sentence.replace('.', '')

        sentimentValue = float(sentenceAnalysis['sentimentValue'])

        sentiments += [sentimentValue]
    return numpy.average(sentiments)

## Scrape web for news article

In [245]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
from newsapi import NewsApiClient

def getPageContent(url):
    try: 
        with closing(get(url, stream = True)) as page:
            return page.content.decode("utf-8")
    except RequestException as e:
        print(e)
        return
    
def parseHTML(url):
    return BeautifulSoup(getPageContent(url), 'html.parser')

# Extracts the text from a CNN article with given URL, excluding the headline and any advertisements.
def getCNNText(url):
    htmlParser = parseHTML(url)
    
    text = ""

    for element in htmlParser.select('div'):
        if element.has_attr('class') and 'zn-body__paragraph' in element['class']: 
            text += element.text 
    return text

# Uses NewsAPI to extract all articles from a given day.
def getArticleURLs(day, month, year):
    dayPrefix = "%04d-%02d-%02d" % (year, month, day)
    
    newsapi = NewsApiClient(api_key = '8d99d69a251a453f8c084f4768db7195')
    
    urls = []
    second = 0
    minute = 0
    for hour in range (0, 24):
        for minute in range (0, 59, 10):
            iso8601DateStart = "%sT%s" % (dayPrefix, "%02d:%02d:%02d" % (hour, minute, second))
            iso8601DateEnd = "%sT%s" % (dayPrefix, "%02d:%02d:%02d" % (hour, minute + (10 if minute != 50 else 9), second))
            all_articles = newsapi.get_everything(q='bitcoin', from_param= iso8601DateStart, to=iso8601DateEnd, page=1)
            for article in all_articles['articles']:
                urls += article['url']
    return urls

## Kill the NLP server

In [247]:
exec('kill $(lsof -ti tcp:9000)')